In [ ]:
%pip install -q redis "redisvl>=0.4.1" langchain-community pypdf sentence-transformers langchain pandas==2.1.4

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.4 which is incompatible.
mizani 0.13.3 requires pandas>=2.2.0, but you have pandas 2.1.4 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 2.1.4 which is incompatible.


In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failed writing body


In [ ]:
# Import necessary libraries
import os
import re
from langchain.vectorstores import Redis
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings
from redisvl.utils.vectorize import HFTextVectorizer
from langchain.embeddings import SentenceTransformerEmbeddings
import pandas as pd
import time
import psutil
import shutil
import nltk
# Download NLTK resources for text processing
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Set up Redis URL
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

In [ ]:
# Define a utility function to clean text (lowercase, remove extra spaces)
def simple_clean(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Load the Excel file containing resume texts and summaries
df = pd.read_excel('/content/synthetic_resume_summaries (1).xlsx')
df.head()

,text,summary
0,Medical Advisor - General Medicines (Diabetes)...,**Medical Advisor** with over 15 years of expe...
1,"linkedin.com/in/hafsah09/\n\nSUMMARY ,\n* Over...",**Digital Product Management Professional** wi...
2,"'o\n\n+\n\nSummary ,\nI have a very positive a...",**Accountant** with experience in financial ma...
3,BRIDGETTE\nWENG\n2007\n\nPROFILE\n\nExtremely ...,**Medical Assistant** with hands-on experience...
4,"Professional Summary ,\nA highly organised, mo...",**Aspiring Legal Professional** with a strong ...


In [ ]:
# Extract texts (full resumes) and summaries
texts = df["text"].tolist()
summaries = df["summary"].tolist()
N = len(texts)

# Clean texts and summaries to standardize them
texts_clean = [simple_clean(text) for text in texts]
summaries_clean = [simple_clean(str(summary)) for summary in summaries]

# Create documents with metadata (ID) for ChromaDB to track which text corresponds to which summary
docs = [Document(page_content=text, metadata={"id": i}) for i, text in enumerate(texts_clean)]

In [ ]:
# Define custom embedding class using HFTextVectorizer
class HFEmbeddings(Embeddings):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.hf = HFTextVectorizer(model_name)

    def embed_documents(self, texts):
        return self.hf.embed_many(texts)

    def embed_query(self, text):
        return self.hf.embed(text)

In [ ]:
# Create embeddings
embeddings = HFEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import numpy as np
print(np.__version__)  # Check the installed version
print(dir(np))

2.0.2
['False_', 'ScalarType', 'True_', '_CopyMode', '_NoValue', '__NUMPY_SETUP__', '__all__', '__array_api_version__', '__builtins__', '__cached__', '__config__', '__dir__', '__doc__', '__expired_attributes__', '__file__', '__former_attrs__', '__future_scalars__', '__getattr__', '__loader__', '__name__', '__numpy_submodules__', '__package__', '__path__', '__spec__', '__version__', '_core', '_distributor_init', '_expired_attrs_2_0', '_get_promotion_state', '_globals', '_int_extended_msg', '_mat', '_msg', '_no_nep50_warning', '_pyinstaller_hooks_dir', '_pytesttester', '_set_promotion_state', '_specific_msg', '_type_info', '_typing', '_utils', 'abs', 'absolute', 'acos', 'acosh', 'add', 'all', 'allclose', 'amax', 'amin', 'angle', 'any', 'append', 'apply_along_axis', 'apply_over_axes', 'arange', 'arccos', 'arccosh', 'arcsin', 'arcsinh', 'arctan', 'arctan2', 'arctanh', 'argmax', 'argmin', 'argpartition', 'argsort', 'argwhere', 'around', 'array', 'array2string', 'array_equal', 'array_equiv',

In [ ]:
# Measure ingestion time
start_time = time.time()
vectorstore = Redis.from_documents(
    documents=docs,
    embedding=embeddings,
    redis_url=REDIS_URL,
    index_name="my_redis_index"
)
ingestion_time = time.time() - start_time
print(f"Ingestion time: {ingestion_time} seconds")

Ingestion time: 6.050894737243652 seconds


In [ ]:
# Set up retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [ ]:
# Initialize lists for performance metrics
latencies = []
recalls = []
# Loop through each summary as query (assuming summaries_clean is defined)
for i, query in enumerate(summaries_clean):
    # Measure latency
    start_time = time.time()
    retrieved_docs = retriever.get_relevant_documents(query)
    latency = time.time() - start_time
    latencies.append(latency)

    # Check recall: if the correct document (ID=i) is retrieved
    retrieved_ids = [doc.metadata.get("id") for doc in retrieved_docs]
    correct_id = i
    if correct_id in retrieved_ids:
        recalls.append(1)
    else:
        recalls.append(0)

# Calculate metrics
average_recall = sum(recalls) / len(recalls)
print(f"Average recall@1: {average_recall}")

Average recall@1: 0.786


In [ ]:
average_latency = sum(latencies) / len(latencies)
print(f"Average latency: {average_latency} seconds")

Average latency: 0.008537956237792969 seconds


In [ ]:
total_queries = len(summaries_clean)
total_time = sum(latencies)
QPS = total_queries / total_time if total_time > 0 else 0
print(f"QPS: {QPS}")

QPS: 117.12404844306117


In [ ]:
# Memory usage (system-wide)
memory_usage = psutil.virtual_memory().used / (1024 ** 3)
print(f"Memory usage: {memory_usage} GB")

Memory usage: 2.131397247314453 GB
